<center><h1> Next-day Stock Price Forecasting using LSTM model with Tensorflow/Keras</h1></center>  

Goal: Predict next-day close price for AAPL stock (daily regression)
Dataset: Daily OHLCV from Yahoo Finance, last 5 years (~1250 rows). 


In [2]:
import yfinance as yf
import pandas as pd

import numpy as np
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

2025-10-17 09:25:53.948343: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Loading data for last 5 years from yahoo finance:
ticker = "AAPL"
df_tfk = yf.download(ticker, period="5y", interval="1d", auto_adjust=False)
df_tfk = df_tfk.rename(columns={'Adj Close': 'Adj_Close'})
# Keep only the second level (the actual data columns)
df_tfk.columns = [col[0] if col[1] == 'AAPL' else col[1] for col in df_tfk.columns]
df_tfk = df_tfk.sort_index()

[*********************100%***********************]  1 of 1 completed


In [6]:
# Create the Target column
df_tfk['Target'] = df_tfk['Adj_Close'].shift(-1)
df_tfk = df_tfk.dropna()

# Defineing features (X) and target (y)
features = ['Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
target = 'Target'

X = df_tfk[features]
y = df_tfk[target]

# Splitting data
train_size = int(len(df_tfk) * 0.8)
val_size = int(len(df_tfk) * 0.1)

X_train = X.iloc[:train_size].values
y_train = y.iloc[:train_size].values

X_val = X.iloc[train_size:train_size + val_size].values
y_val = y.iloc[train_size:train_size + val_size].values

X_test = X.iloc[train_size + val_size:].values
y_test = y.iloc[train_size + val_size:].values

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

Train: 1004, Val: 125, Test: 126


In [8]:
# Scaling
# Initialize scalers
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit only on training data
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))

# Transform validation and test using same scaler
X_val_scaled = scaler_X.transform(X_val)
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1))
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))


### Create Sequences for LSTM   
Each sample = last 60 days → predict next day.

In [11]:
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 60

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)

#### Above function reorganize the data: f.e. for X_train_seq (944, 60, 6) - 944 sequences (1004-60), 60 - time step, and 6 - features.¶    


## Build LSTM model¶

In [15]:

model_lstm = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, len(features))),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.summary()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 64)         │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,137 (121.63 KB)

 Trainable params: 31,137 (121.63 KB)

 Non-trainable params: 0 (0.00 B)

#### Parameters in neural networks:  
Parameters = weights + biases that the model learns during training.  
Every layer in the network has:  
Weights — the learned multipliers for inputs.  
Biases — the learned offset values added after multiplying inputs by weights.  
Together, these control how the model transforms input into output.  
In total we have 31137 parameters

### ### Train and test the model with validation set

In [19]:

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model_lstm.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=60,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0714 - val_loss: 0.0124
Epoch 2/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0086 - val_loss: 0.0133
Epoch 3/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0077 - val_loss: 0.0094
Epoch 4/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0059 - val_loss: 0.0092
Epoch 5/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0059 - val_loss: 0.0100
Epoch 6/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0050 - val_loss: 0.0090
Epoch 7/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0047 - val_loss: 0.0117
Epoch 8/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0039 - val_loss: 0.0129
Epoch 9/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0046 - val_loss: 0.0126
Epoch 10/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 0.0041 - val_loss: 0.0092
Epoch 11/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0044 - val_loss: 0.0095
Epoch 12/60
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0

### Evaluation

In [22]:

# Predict on test set
y_pred_scaled = model_lstm.predict(X_test_seq)

# Inverse scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_inv = scaler_y.inverse_transform(y_test_seq)

rmse = sqrt(mean_squared_error(y_test_inv, y_pred))
mae = mean_absolute_error(y_test_inv, y_pred)

print(f"Test RMSE: {rmse:.3f}")
print(f"Test MAE: {mae:.3f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step
Test RMSE: 9.301
Test MAE: 7.723


Test RMSE: 7.699  
Test MAE: 6.117

In [24]:
model_lstm.save("best_lstm_model.h5")